# Imports

In [6]:
import os
import sys
import json
from datetime import datetime
import subprocess
import pandas as pd
import plotly.graph_objects as go
current_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))
sys.path.append(root_dir)
import Birdeye.Basics.Master_Functions as Master_Functions
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Birdeye Data Processing

In [7]:
# Use the already defined root_dir to create absolute paths
python_file_path = os.path.join(root_dir, 'Bots', 'Fib', 'Dataset_Generator_Rugcheck_Fib.py')

# Set PYTHONPATH environment variable to include root_dir
env = os.environ.copy()
env['PYTHONPATH'] = root_dir

# Run the Python file with the modified environment
subprocess.run(['python', python_file_path], env=env)

{"success":true,"data":["solana","ethereum","arbitrum","avalanche","bsc","optimism","polygon","base","zksync"]}

--------------------------------

Running on: solana

--------------------------------

NEW TOKEN FILTERS:

Min Liquidity: 20K
Max Liquidity: 200K
Min Market Cap: 200K
Max Market Cap: 5.00Mil
Filtering New Tokens for: 1 days, 12 hours, 0 minutes back
Token Launch Liquidity Filter: 20K
RugCheck Risk Score Threshold: 1000

--------------------------------
 
 
--------------------------------


Performing RugCheck Assessment...


ERROR:rugcheck_monitor:Error fetching data for 913i59SfHgt7XgxG9WjLJ6FeYRkRhKme69Ywp47Y2mFM: 429, message='Too Many Requests', url='https://api.rugcheck.xyz/v1/tokens/913i59SfHgt7XgxG9WjLJ6FeYRkRhKme69Ywp47Y2mFM/report/summary'
ERROR:rugcheck_monitor:Error fetching data for 6xTuFTZFC2KNjkrMNi7vgGzFJpvAQtCEh7jHq7gLvK16: 429, message='Too Many Requests', url='https://api.rugcheck.xyz/v1/tokens/6xTuFTZFC2KNjkrMNi7vgGzFJpvAQtCEh7jHq7gLvK16/report/summary'
ERROR:rugcheck_monitor:Error fetching data for 8B9MyZMxRjsSXtoXiw5J6VqKkRASDBJy2KmBUzbwMUpx: 429, message='Too Many Requests', url='https://api.rugcheck.xyz/v1/tokens/8B9MyZMxRjsSXtoXiw5J6VqKkRASDBJy2KmBUzbwMUpx/report/summary'
ERROR:rugcheck_monitor:Error fetching data for CNTF58JYR8Sb1L6oqhchVp1FjXus5LnS1j6iX2B5ywdv: 429, message='Too Many Requests', url='https://api.rugcheck.xyz/v1/tokens/CNTF58JYR8Sb1L6oqhchVp1FjXus5LnS1j6iX2B5ywdv/report/summary'
ERROR:rugcheck_monitor:Error fetching data for HNSb2Mr3uRfeEkUVnxPAhNhwhRGYbmKnGRZxsQyVh


Before filtering:
Total tokens: 261

Sample of data:
                                                  Liquidity  ...  Risk_Status
Address                                                      ...             
DGwwy32VYVGtBBWusrPtAYU1jJuMja1DtMC6fCp74qrm  391379.123321  ...    HIGH RISK
BW1rpYbk4uFKqhFgWqWBce4GynGXt98Y9KD581fgt4ey  242556.840078  ...    HIGH RISK
6tJxXcG5JfQr6NZk5z1CPyQAU2vXgRYbocrxdyKpump   194537.675494  ...    HIGH RISK
79svjscsNsidW89HYW7K62NNA6g7xBcmAQFKHgqp9fXv  871547.920473  ...    HIGH RISK
3RaegXi9YhRAEUozshMB8Usb8oT8fxfd9mRQ9Wa3GSKw   38814.933625  ...    HIGH RISK

[5 rows x 4 columns]

After filtering:
Remaining tokens: 10

Filtered tokens data:

Token: Cj9asszFUhxJ39zdxGNP4ocqaEtVqSWHqEdDdgK4jFeo
Liquidity: 30901.27348000472
Market Cap: 214252.7780905324
RugCheck Score: 500.0

Token: 9cYXqd1ggyMTjwZFWnbDuf5EfLEcLuumMG67jRkJpump
Liquidity: 54386.59120944461
Market Cap: 225038.26767371735
RugCheck Score: 400.0

Token: Ab1TGzvprBRCRixU2cAHQU91TfqQbntQF4Ev9Ld

CompletedProcess(args=['python', '/home/ubuntu/Birdeye_Dashboard/Bots/Fib/Dataset_Generator_Rugcheck_Fib.py'], returncode=0)

### Latest Data Import


In [8]:
# Get the most recent folder
base_path = os.path.join(current_dir, 'Data', 'New_Token_Data')
current_date = datetime.now().strftime('%Y_%m_%d')
date_folder = os.path.join(base_path, current_date)
ohlcv_folder = os.path.join(date_folder, 'OHLCV_Data')
ohlcv_datetime_folder = Master_Functions.get_most_recent_folder(ohlcv_folder)

# Print the name of the folder being used for import
print(f"Importing OHLCV data from folder: {ohlcv_datetime_folder}")

# Import the OHLCV data
imported_ohlcv_data = Master_Functions.import_ohlcv_data(ohlcv_datetime_folder)

Importing OHLCV data from folder: /home/ubuntu/Birdeye_Dashboard/Bots/Fib/Execution_Algo/Data/New_Token_Data/2024_11_29/OHLCV_Data/2024_Nov_29_0322AM


### Plotting Fib Levels

In [21]:
def plot_price_and_fib_levels(imported_ohlcv_data, fib_levels, initial_timeframe='5m', target_fib_level=0.786, min_market_cap=0):
    """
    Plot price action with Fibonacci levels using a dark theme and enhanced display features.
    Identifies and plots tokens in their first descent from ATH to specified Fibonacci levels.
    
    Args:
        imported_ohlcv_data (dict): Dictionary containing OHLCV data for multiple tokens and timeframes
        fib_levels (list): List of Fibonacci levels to plot (e.g., [0.236, 0.382, ...])
        initial_timeframe (str): Default timeframe to display ('1m', '5m', '15m', '1H')
        target_fib_level (float): The specific Fibonacci level to check for first descent
        min_market_cap (float): Minimum market cap threshold for token inclusion
    """
    def calculate_fib_proximity(df, level=0.786, fib_levels=[0.236, 0.382, 0.5, 0.618, 0.786, 0.886]):
        """
        Calculate proximity to Fibonacci level and determine if token is in first descent.
        Also checks if price is between target fib level and next higher level.
        
        Args:
            df (pd.DataFrame): OHLCV data for a token
            level (float): Target Fibonacci level to check against
            fib_levels (list): List of all Fibonacci levels in ascending order
            
        Returns:
            tuple: (proximity_percentage, fib_price, current_price, is_first_descent)
                - proximity_percentage: % distance from current price to fib level
                - fib_price: Price at the specified fib level
                - current_price: Current token price
                - is_first_descent: Boolean indicating if this is the first descent to this level
        """
        if df.empty:
            return float('inf'), 0, 0, False
            
        # Get the next higher fib level
        current_level_index = fib_levels.index(level)
        next_level = fib_levels[current_level_index - 1] if current_level_index > 0 else None
            
        ath_close = df['close'].max()
        ath_idx = df['close'].idxmax()
        atl = df['low'].min()
        current_price = df['close'].iloc[-1]
        current_low = df['low'].iloc[-1]
        current_open = df['open'].iloc[-1]
        
        # Calculate prices for both fib levels
        target_fib_price = ath_close - (ath_close - atl) * level
        next_fib_price = ath_close - (ath_close - atl) * next_level if next_level else float('inf')
        
        # Get all data after ATH close value is first reached
        first_ath_idx = df[df['close'] >= ath_close].index[0]
        df_after_ath = df.loc[first_ath_idx:]
        
        # Check if any previous candle has breached the target fib level
        has_previous_breach = any((df_after_ath['open'] < target_fib_price) | (df_after_ath['high'] < target_fib_price))
        
        # Check if current price is between the two fib levels
        is_between_levels = (
            max(current_price, current_low, current_open) <= next_fib_price and 
            min(current_price, current_low, current_open) > target_fib_price
        )
        
        # Calculate proximity only if price is above fib level, no previous breach, and between levels
        if current_price > target_fib_price and not has_previous_breach and is_between_levels:
            proximity = abs((current_price - target_fib_price) / target_fib_price * 100)
        else:
            proximity = float('inf')
            
        return proximity, target_fib_price, current_price, not has_previous_breach and is_between_levels
    
    def create_timeframe_annotations(timeframe, market_cap, volume):
        """Create annotations for timeframe, market cap, and volume display."""
        formatted_volume = Master_Functions.format_number(volume)
        return [
            dict(
                text=f"<b>Timeframe:</b> {timeframe}",
                x=0.2,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Current MC:</b> {market_cap}",
                x=0.5,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            ),
            dict(
                text=f"<b>Prior Candle Vol:</b> {formatted_volume}",
                x=0.8,
                y=-0.07,
                xref="paper",
                yref="paper",
                showarrow=False,
                font=dict(size=16, color=TITLE_COLOR),
                bgcolor=BACKGROUND_COLOR,
                bordercolor=GRID_COLOR,
                borderwidth=1,
                borderpad=8
            )
        ]
        
        
    def has_outliers(df, threshold_multiplier=15.0):
        """
        Check if the price data contains outliers that might skew the analysis.
        
        Args:
            df (pd.DataFrame): OHLCV data
            threshold_multiplier (float): Multiplier to determine outlier threshold
            
        Returns:
            bool: True if outliers detected, False otherwise
        """
        if df.empty:
            return True
            
        median_price = df['close'].median()
        high_outliers = df['high'] > (median_price * threshold_multiplier)
        
        if not high_outliers.any():
            return False
            
        outlier_indices = df.index[high_outliers]
        for idx in outlier_indices:
            start_idx = max(0, df.index.get_loc(idx) - 3)
            end_idx = min(len(df), df.index.get_loc(idx) + 4)
            window = df.iloc[start_idx:end_idx]
            
            window_median = window['close'].median()
            if df.loc[idx, 'high'] > window_median * threshold_multiplier:
                return True
                
        return False
    
    # Define color scheme with fainter grid
    BACKGROUND_COLOR = '#1e1e1e'
    TEXT_COLOR = '#ffffff'
    GRID_COLOR = 'rgba(51, 51, 51, 0.3)'  # Fainter grid lines
    TITLE_COLOR = '#00cc00'
    FIB_COLORS = ['#0077BE',  # Neon deep ocean blue for 0.236
                  '#ff9100',   # Current orange for 0.382
                  '#FFD700',   # Neon golden yellow for 0.5
                  '#00FF7F',   # Dark neon green for 0.618
                  '#FF00FF',   # Neon pink purple for 0.786
                  '#FFFDD0']   # Neon cream for 0.886
    ATH_COLOR = '#FFD700'     # Neon yellow gold for ATH star
    
    available_timeframes = ['1m', '5m', '15m', '1H']
    token_data = []
    
    # Process market cap data
    base_path = 'Data/New_Token_Data'
    date_folders = [f for f in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, f)) and f.startswith('2024_')]
    most_recent_date = max(date_folders)
    date_folder = os.path.join(base_path, most_recent_date)
    
    summary_folder = os.path.join(date_folder, 'Token_Summary')
    datetime_folders = [f for f in os.listdir(summary_folder) if os.path.isdir(os.path.join(summary_folder, f))]
    most_recent_datetime = max(datetime_folders)
    summary_datetime_folder = os.path.join(summary_folder, most_recent_datetime)
    
    mc_files = [f for f in os.listdir(summary_datetime_folder) if f.startswith('new_tokens_mc') and f.endswith('.csv')]
    latest_mc_file = os.path.join(summary_datetime_folder, mc_files[0])
    market_caps = pd.read_csv(latest_mc_file, index_col=0)['Market Cap']
    

    # Calculate and sort tokens
    for token_address, timeframes in imported_ohlcv_data.items():
        if initial_timeframe not in timeframes:
            continue
            
        df_initial = timeframes[initial_timeframe]
        if has_outliers(df_initial):
            continue
            
        # Get current market cap for the token
        current_market_cap = market_caps.get(token_address, 0)
        
        # Skip tokens below minimum market cap threshold
        if current_market_cap < min_market_cap:
            continue
        
        # Check for first descent pattern
        proximity, fib_price, current_price, is_first_descent = calculate_fib_proximity(
            df_initial,
            level=target_fib_level           # Use the specified target level
        )
        
        # Only add tokens in their first descent and with valid proximity
        if is_first_descent and proximity != float('inf'):
            token_data.append({
                'address': token_address,
                'proximity': proximity,
                'fib_price': fib_price,
                'current_price': current_price
            })
    
    # Sort tokens by proximity to fib level
    sorted_tokens = sorted(token_data, key=lambda x: x['proximity'])
    
    # Create plots for each token
    for token_data in sorted_tokens:
        token_address = token_data['address']
        gmgn_link = f"https://gmgn.ai/sol/token/{token_address}"
        dexscreener_link = f"https://dexscreener.com/solana/{token_address}"
        print(f"\nProcessing: {gmgn_link}\n{dexscreener_link}")
        
        timeframes = imported_ohlcv_data[token_address]
        market_cap = Master_Functions.format_number(market_caps.get(token_address, 0))
        
        fig = go.Figure(layout_xaxis_rangeslider_visible=False)
        timeframe_traces = {}
        timeframe_volumes = {}
        
        # Process each timeframe
        for tf in available_timeframes:
            if tf not in timeframes:
                continue
                
            df = timeframes[tf]
            if df.empty:
                continue
                
            traces = []
            timeframe_volumes[tf] = df['volume'].iloc[-1]
            
            # Calculate ATH close and ATL
            ath_close = df['close'].max()
            ath_idx = df['close'].idxmax()
            atl = df['low'].min()
            
            # Add candlestick chart
            traces.append(go.Candlestick(
                x=df.index,
                open=df['open'],
                high=df['high'],
                low=df['low'],
                close=df['close'],
                name="Price",
                showlegend=False,
                increasing_line_color='#00ff9d',
                decreasing_line_color='#ff005b',
                visible=(tf == initial_timeframe)
            ))
            
            # Add ATH star with glow effect
            traces.append(go.Scatter(
                x=[ath_idx],
                y=[ath_close],
                mode='markers',
                marker=dict(
                    symbol='star',
                    size=24,
                    color=ATH_COLOR,
                    line=dict(
                        color=ATH_COLOR,
                        width=2
                    )
                ),
                name='ATH Close',
                showlegend=False,
                visible=(tf == initial_timeframe)
            ))
            
            # Add Fibonacci level lines (thicker for mobile)
            for i, level in enumerate(fib_levels):
                fib_price = ath_close - (ath_close - atl) * level
                traces.append(go.Scatter(
                    x=[df.index[0], df.index[-1]],
                    y=[fib_price, fib_price],
                    mode='lines',
                    line=dict(color=FIB_COLORS[i], width=3),  # Thicker lines
                    showlegend=False,
                    visible=(tf == initial_timeframe)
                ))
            
            timeframe_traces[tf] = traces
        
        # Create mobile-friendly timeframe buttons
        buttons = []
        for tf in available_timeframes:
            if tf not in timeframe_traces:
                continue
                
            visible_traces = []
            for other_tf in timeframe_traces:
                visible_traces.extend([tf == other_tf] * len(timeframe_traces[other_tf]))
            
            tf_annotations = create_timeframe_annotations(
                tf,
                market_cap,
                timeframe_volumes[tf]
            )
            
            buttons.append(dict(
                label=tf,
                method="update",
                args=[
                    {"visible": visible_traces},
                    {"annotations": tf_annotations}
                ]
            ))
        
        # Add all traces to the figure
        for traces in timeframe_traces.values():
            fig.add_traces(traces)
        
        # Create initial annotations
        initial_annotations = create_timeframe_annotations(
            initial_timeframe,
            market_cap,
            timeframe_volumes[initial_timeframe]
        )
        
        # Update layout with fixed dimensions
        fig.update_layout(
            height=800,
            width=1200,
            paper_bgcolor=BACKGROUND_COLOR,
            plot_bgcolor=BACKGROUND_COLOR,
            margin=dict(r=50, t=80, l=50, b=50),
            showlegend=False,
            hovermode='x unified',
            annotations=initial_annotations,
            updatemenus=[dict(
                type="buttons",
                direction="right",
                x=0.5,
                y=1.05,
                xanchor="center",
                yanchor="bottom",
                showactive=True,
                active=available_timeframes.index(initial_timeframe),
                buttons=buttons,
                bgcolor=BACKGROUND_COLOR,
                bordercolor="#555555",
                borderwidth=2,
                font=dict(
                    size=16,  # Larger font for buttons
                    color=TITLE_COLOR
                ),
                pad=dict(r=20, t=10, b=10, l=20)  # Increased padding on all sides
            )],
            xaxis=dict(
                showticklabels=False,
                showgrid=True,
                gridcolor=GRID_COLOR,
                showline=True,
                linecolor=GRID_COLOR,
                rangeslider=dict(visible=False)
            ),
            yaxis=dict(
                showgrid=True,
                gridcolor=GRID_COLOR,
                showticklabels=False,
                showline=False,
                zeroline=False,
                fixedrange=True
            )
        )
        
        fig.show()

# Define Fibonacci levels (keep all levels for display, but we'll specify which one to check)
fib_levels = [0.236, 0.382, 0.5, 0.618, 0.786, 0.886]

# Call the function with configurable parameters
plot_price_and_fib_levels(
    imported_ohlcv_data=imported_ohlcv_data,
    fib_levels=fib_levels,
    initial_timeframe='15m',  # Easily change timeframe here
    target_fib_level=0.786,   # Specify which fib level to check for first descent
    min_market_cap=1500000     # Add minimum market cap threshold
)


Processing: https://gmgn.ai/sol/token/waGerWB7pKNvEfvQmhA2eS61Vs45BsLAmpFff46Lc3v
https://dexscreener.com/solana/waGerWB7pKNvEfvQmhA2eS61Vs45BsLAmpFff46Lc3v
